In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from tqdm import tqdm
import time

train = pd.read_csv("CSV_train.csv",low_memory=False,delimiter=';')
test=pd.read_csv("CSV_test.csv",low_memory=False,delimiter=',')
hidden=pd.read_csv("CSV_hidden_test.csv",low_memory=False,delimiter=',')

In [2]:
# storing length of datasets 
train_len = train.shape[0] 
test_len = test.shape[0]
All_data = pd.concat((train,test,hidden)).reset_index(drop=True) 

lithology_keys = {30000: 'Sandstone',
                 65030: 'Sandstone/Shale',
                 65000: 'Shale',
                 80000: 'Marl',
                 74000: 'Dolomite',
                 70000: 'Limestone',
                 70032: 'Chalk',
                 88000: 'Halite',
                 86000: 'Anhydrite',
                 99000: 'Tuff',
                 90000: 'Coal',
                 93000: 'Basement'}
All_data['Lithology'] = All_data['FORCE_2020_LITHOFACIES_LITHOLOGY'].map(lithology_keys)
All_data

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0,Shale
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0,Shale
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0,Shale
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0,Shale
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0,Shale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,35/9-7,2973.2988,536096.06250,6793022.0,-2943.444580,BAAT GP.,Etive Fm.,8.276272,NaN,2.820439,...,136.911575,NaN,0.502458,NaN,2.311106,24.306124,NaN,65000,2.0,Shale
1429690,35/9-7,2973.4508,536096.06250,6793022.0,-2943.595947,BAAT GP.,Etive Fm.,8.267273,NaN,3.020778,...,137.583923,NaN,0.374753,NaN,1.853418,22.201078,NaN,65000,2.0,Shale
1429691,35/9-7,2973.6028,536096.06250,6793022.0,-2943.747559,BAAT GP.,Etive Fm.,8.250099,NaN,2.795711,...,138.310898,NaN,0.211487,NaN,1.325961,20.096741,NaN,65000,2.0,Shale
1429692,35/9-7,2973.7548,536096.06250,6793022.0,-2943.899170,BAAT GP.,Etive Fm.,NaN,NaN,2.658694,...,137.592819,NaN,0.147950,NaN,1.260347,17.992323,NaN,65000,2.0,Shale


In [3]:
#dropping columns with high missing values
drop_cols = ['SGR', 'ROPA', 'RXO', 'MUDWEIGHT','DCAL','RMIC','FORCE_2020_LITHOFACIES_CONFIDENCE']
All_data_drop = All_data.drop(drop_cols, axis=1)
# encoding categorical variables
All_data_drop['GROUP_encoded'] = All_data_drop['GROUP'].astype('category')
All_data_drop['GROUP_encoded'] = All_data_drop['GROUP_encoded'].cat.codes

All_data_drop['FORMATION_encoded'] = All_data_drop['FORMATION'].astype('category')
All_data_drop['FORMATION_encoded'] = All_data_drop['FORMATION_encoded'].cat.codes

All_data_drop['WELL_encoded'] = All_data_drop['WELL'].astype('category')
All_data_drop['WELL_encoded'] = All_data_drop['WELL_encoded'].cat.codes

All_data_drop['Lithology_encoded'] = All_data_drop['FORCE_2020_LITHOFACIES_LITHOLOGY'].astype('category')
All_data_drop['Lithology_encoded'] = All_data_drop['Lithology_encoded'].cat.codes

In [4]:
All_data_drop

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,BS,ROP,DTS,DRHO,FORCE_2020_LITHOFACIES_LITHOLOGY,Lithology,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,34.636410,NaN,-0.574928,65000,Shale,6,-1,1,1
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,34.636410,NaN,-0.570188,65000,Shale,6,-1,1,1
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,34.779556,NaN,-0.574245,65000,Shale,6,-1,1,1
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,39.965164,NaN,-0.586315,65000,Shale,6,-1,1,1
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,57.483765,NaN,-0.597914,65000,Shale,6,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,35/9-7,2973.2988,536096.06250,6793022.0,-2943.444580,BAAT GP.,Etive Fm.,8.276272,NaN,2.820439,...,8.5,15.195305,136.911575,0.502458,65000,Shale,0,16,113,1
1429690,35/9-7,2973.4508,536096.06250,6793022.0,-2943.595947,BAAT GP.,Etive Fm.,8.267273,NaN,3.020778,...,8.5,15.770223,137.583923,0.374753,65000,Shale,0,16,113,1
1429691,35/9-7,2973.6028,536096.06250,6793022.0,-2943.747559,BAAT GP.,Etive Fm.,8.250099,NaN,2.795711,...,8.5,16.418465,138.310898,0.211487,65000,Shale,0,16,113,1
1429692,35/9-7,2973.7548,536096.06250,6793022.0,-2943.899170,BAAT GP.,Etive Fm.,NaN,NaN,2.658694,...,8.5,17.037945,137.592819,0.147950,65000,Shale,0,16,113,1


In [5]:
#dropping categorial features replaces beforehan by encoded features
drop2 = All_data_drop.drop(['GROUP', 'FORMATION','WELL','FORCE_2020_LITHOFACIES_LITHOLOGY','Lithology'], axis=1)

# splitting dataset into training, test, and hidden sets
train_prep = drop2[:train_len].copy()
test_prep = drop2[train_len:(train_len+test_len)].copy()
hidden_prep = drop2[(train_len+test_len):].copy()

In [6]:
drop2

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,NaN,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,NaN,34.636410,NaN,-0.574928,6,-1,1,1
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,NaN,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,NaN,34.636410,NaN,-0.570188,6,-1,1,1
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,NaN,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,NaN,34.779556,NaN,-0.574245,6,-1,1,1
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,NaN,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,NaN,39.965164,NaN,-0.586315,6,-1,1,1
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,NaN,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,NaN,57.483765,NaN,-0.597914,6,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.2988,536096.06250,6793022.0,-2943.444580,8.276272,NaN,2.820439,3.158570,NaN,90.720284,...,75.260658,NaN,8.5,15.195305,136.911575,0.502458,0,16,113,1
1429690,2973.4508,536096.06250,6793022.0,-2943.595947,8.267273,NaN,3.020778,3.332977,NaN,87.062027,...,74.868301,NaN,8.5,15.770223,137.583923,0.374753,0,16,113,1
1429691,2973.6028,536096.06250,6793022.0,-2943.747559,8.250099,NaN,2.795711,3.044179,NaN,86.115921,...,74.848122,NaN,8.5,16.418465,138.310898,0.211487,0,16,113,1
1429692,2973.7548,536096.06250,6793022.0,-2943.899170,NaN,NaN,2.658694,2.847681,NaN,89.497131,...,74.964027,NaN,8.5,17.037945,137.592819,0.147950,0,16,113,1


In [7]:
train_prep1= train_prep.copy()
test_prep1= test_prep.copy()
hidden_prep1= hidden_prep.copy()

In [8]:
#Inputing missing values by introducing median 
from sklearn.impute import SimpleImputer
miss = SimpleImputer(missing_values=np.nan, strategy='median')
miss.fit(drop2)
All_imp = miss.fit_transform(drop2)
All_imp=pd.DataFrame(All_imp, columns=['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO',
         'GROUP_encoded',
       'FORMATION_encoded', 'WELL_encoded','Lithology_encoded'])
All_imp

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,1.398049,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,12.250001,34.636410,189.362198,-0.574928,6.0,-1.0,1.0,1.0
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,1.398049,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,12.250001,34.636410,189.362198,-0.570188,6.0,-1.0,1.0,1.0
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,1.398049,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,12.250001,34.779556,189.362198,-0.574245,6.0,-1.0,1.0,1.0
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,1.398049,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,12.250001,39.965164,189.362198,-0.586315,6.0,-1.0,1.0,1.0
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,1.398049,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,12.250001,57.483765,189.362198,-0.597914,6.0,-1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.2988,536096.06250,6793022.0,-2943.444580,8.276272,1.398049,2.820439,3.158570,2.331407,90.720284,...,75.260658,54.270451,8.500000,15.195305,136.911575,0.502458,0.0,16.0,113.0,1.0
1429690,2973.4508,536096.06250,6793022.0,-2943.595947,8.267273,1.398049,3.020778,3.332977,2.331407,87.062027,...,74.868301,54.270451,8.500000,15.770223,137.583923,0.374753,0.0,16.0,113.0,1.0
1429691,2973.6028,536096.06250,6793022.0,-2943.747559,8.250099,1.398049,2.795711,3.044179,2.331407,86.115921,...,74.848122,54.270451,8.500000,16.418465,138.310898,0.211487,0.0,16.0,113.0,1.0
1429692,2973.7548,536096.06250,6793022.0,-2943.899170,12.515673,1.398049,2.658694,2.847681,2.331407,89.497131,...,74.964027,54.270451,8.500000,17.037945,137.592819,0.147950,0.0,16.0,113.0,1.0


In [9]:
print(All_imp['GR'].quantile(0.50)) 
print(All_imp['GR'].quantile(0.95)) 
All_imp['GR'] = np.where(All_imp['GR'] > 150, 67, All_imp['GR'])
All_imp['GR'].describe()

67.860420227
124.80774611999996


count    1.429694e+06
mean     6.765757e+01
std      2.796972e+01
min      1.092843e-01
25%      4.737605e+01
50%      6.700000e+01
75%      8.707136e+01
max      1.499989e+02
Name: GR, dtype: float64

In [10]:
train_imp = All_imp[:train_len].copy()
test_imp = All_imp[train_len:(train_len+test_len)].copy()
hidden_imp = All_imp[(train_len+test_len):].copy()

In [11]:
print(train_imp.shape)
print(test_imp.shape)
print(hidden_imp.shape)

(1170511, 22)
(136786, 22)
(122397, 22)


In [22]:
train_imp

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,FORMATION_encoded,WELL_encoded,Lithology_encoded,PI,SI,G,K,VSHALE,PHIT,PHIE
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,1.398049,1.611410,1.798681,1.884186,80.200851,...,-1.0,1.0,1.0,11693.489697,9950.168577,5.254570e+07,2.510312e+06,0.534337,1.351844,0.629504
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,1.398049,1.618070,1.795641,1.889794,79.262886,...,-1.0,1.0,1.0,11766.828680,9979.782324,5.270208e+07,2.996897e+06,0.528079,1.355752,0.639807
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,1.398049,1.626459,1.800733,1.896523,74.821999,...,-1.0,1.0,1.0,11840.419324,10015.318694,5.288975e+07,3.402746e+06,0.498452,1.360442,0.682328
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,1.398049,1.621594,1.801517,1.891913,72.878922,...,-1.0,1.0,1.0,11813.422165,9990.974118,5.276119e+07,3.416749e+06,0.485488,1.357229,0.698310
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,1.398049,1.602679,1.795299,1.880034,71.729141,...,-1.0,1.0,1.0,11740.795349,9928.243916,5.242991e+07,3.414604e+06,0.477817,1.348951,0.704399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170506,3169.3124,478648.81250,6737678.5,-2088.677002,8.423170,1.398049,1.455281,1.453520,2.527984,77.654900,...,7.0,117.0,0.0,23340.233544,13349.991148,7.049977e+07,1.214948e+08,0.517351,1.802153,0.869806
1170507,3169.4644,478648.81250,6737678.5,-2088.677002,8.379244,1.398049,1.455281,1.453520,2.537613,75.363937,...,7.0,117.0,2.0,23429.137968,13400.842108,7.076831e+07,1.219576e+08,0.502067,1.808907,0.900714
1170508,3169.6164,478648.81250,6737678.5,-2088.677002,8.350248,1.398049,1.455281,1.453520,2.491860,66.452843,...,7.0,117.0,2.0,23006.712076,13159.225763,6.949236e+07,1.197587e+08,0.442616,1.776819,0.990370
1170509,3169.7684,478648.81250,6737678.5,-2088.677002,8.313779,1.398049,1.455281,1.453520,2.447539,55.784817,...,7.0,117.0,2.0,22597.506941,12925.171339,6.825634e+07,1.176286e+08,0.371444,1.745745,1.097300


In [23]:
train_imp.describe()

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,FORMATION_encoded,WELL_encoded,Lithology_encoded,PI,SI,G,K,VSHALE,PHIT,PHIE
count,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,...,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06,1.170510e+06
mean,2.184087e+03,4.855667e+05,6.681795e+06,-2.138068e+03,1.313538e+01,6.406900e+00,4.869330e+00,1.060410e+01,2.291382e+00,6.792387e+01,...,3.644060e+01,5.752155e+01,1.467345e+00,2.216025e+04,1.215052e+04,6.520212e+07,1.437861e+08,4.524302e-01,1.639302e+00,8.925994e-01
std,9.971825e+02,3.440346e+04,1.276749e+05,9.664754e+02,3.657787e+00,7.401866e+01,5.375808e+01,1.134141e+02,2.357325e-01,2.780841e+01,...,2.394242e+01,3.196307e+01,1.719964e+00,7.872649e+03,2.002256e+03,1.798694e+07,1.886909e+08,1.855260e-01,1.594531e-01,3.087903e-01
min,1.360860e+02,4.268988e+05,6.406641e+06,-5.395563e+03,2.344000e+00,1.000000e-04,-8.418695e-03,3.170056e-02,7.209712e-01,1.092843e-01,...,-1.000000e+00,1.000000e+00,0.000000e+00,5.712704e+03,3.019822e+03,4.463374e+06,-2.992803e+08,0.000000e+00,6.465853e-01,4.240368e-05
25%,1.418597e+03,4.548019e+05,6.593126e+06,-2.804552e+03,9.882804e+00,1.296200e+00,9.290141e-01,9.148610e-01,2.123108e+00,4.762720e+01,...,1.800000e+01,3.400000e+01,1.000000e+00,1.530976e+04,1.102997e+04,5.795326e+07,3.152832e+07,3.170194e-01,1.527993e+00,6.854834e-01
50%,2.076605e+03,4.777699e+05,6.737314e+06,-2.055282e+03,1.251567e+01,1.398049e+00,1.455281e+00,1.449789e+00,2.331407e+00,6.700000e+01,...,3.700000e+01,5.600000e+01,1.000000e+00,2.152529e+04,1.231189e+04,6.501770e+07,1.120473e+08,4.462665e-01,1.664372e+00,8.965746e-01
75%,2.864393e+03,5.201315e+05,6.784877e+06,-1.397963e+03,1.574933e+01,1.510923e+00,2.587101e+00,2.537876e+00,2.462978e+00,8.671846e+01,...,5.800000e+01,8.500000e+01,1.000000e+00,2.749175e+04,1.304067e+04,6.888218e+07,2.094718e+08,5.778197e-01,1.754280e+00,1.096223e+00
max,5.436632e+03,5.726328e+05,6.856661e+06,-1.110860e+02,2.827900e+01,2.193905e+03,1.988616e+03,1.999887e+03,3.457820e+00,1.499953e+02,...,6.900000e+01,1.170000e+02,1.100000e+01,2.626550e+05,3.286750e+04,4.708671e+08,3.534907e+10,9.999757e-01,2.455741e+00,2.226197e+00


In [12]:


# calculating p-impedance
train_imp['PI'] = train_imp.RHOB * (1e6/train_imp.DTC)
test_imp['PI'] = test_imp.RHOB * (1e6/test_imp.DTC)
hidden_imp['PI'] = hidden_imp.RHOB * (1e6/hidden_imp.DTC)

# calculating s-impedance
train_imp['SI'] = train_imp.RHOB * (1e6/train_imp.DTS) 
test_imp['SI'] = test_imp.RHOB * (1e6/test_imp.DTS) 
hidden_imp['SI'] = hidden_imp.RHOB * (1e6/hidden_imp.DTS) 

#calculating Shear modulus (G)
train_imp['G'] = ((1e6/train_imp.DTS)**2) * train_imp.RHOB
test_imp['G'] = ((1e6/test_imp.DTS)**2) * test_imp.RHOB
hidden_imp['G'] = ((1e6/hidden_imp.DTS)**2) * hidden_imp.RHOB

#calculating Bulk modulus (K)
train_imp['K'] = (((1e6/train_imp.DTC)**2) * train_imp.RHOB) - (4 * train_imp.G/3)
test_imp['K'] = (((1e6/test_imp.DTC)**2) * test_imp.RHOB) - (4 * test_imp.G/3)
hidden_imp['K'] = (((1e6/hidden_imp.DTC)**2) * hidden_imp.RHOB) - (4 * hidden_imp.G/3)

# calculate the shale volume
train_imp["VSHALE"] = (train_imp.GR - np.min(train_imp.GR)) / (np.max(train_imp.GR) - np.min(train_imp.GR))
test_imp["VSHALE"] = (test_imp.GR - np.min(test_imp.GR)) / (np.max(test_imp.GR) - np.min(test_imp.GR))
hidden_imp["VSHALE"] = (hidden_imp.GR - np.min(hidden_imp.GR)) / (np.max(hidden_imp.GR) - np.min(hidden_imp.GR))
#train_imp1.head()

# calculate the total porosity
train_imp['PHIT'] = np.sqrt(((((train_imp.NPHI)*(train_imp.NPHI)+(train_imp.RHOB)*(train_imp.RHOB))))/2)
test_imp['PHIT'] = np.sqrt(((((test_imp.NPHI)*(test_imp.NPHI)+(test_imp.RHOB)*(test_imp.RHOB))))/2)
hidden_imp['PHIT'] = np.sqrt(((((hidden_imp.NPHI)*(hidden_imp.NPHI)+(hidden_imp.RHOB)*(hidden_imp.RHOB))))/2)
#train_imp1.tail()

# calculate effective porosity
train_imp['PHIE'] = train_imp.PHIT*(1-train_imp.VSHALE)
train_imp = train_imp[train_imp['PHIE'] !=0]
train_imp['PHIE'] = train_imp['PHIE'].abs() 

test_imp['PHIE'] = test_imp.PHIT*(1-test_imp.VSHALE)
test_imp = test_imp[test_imp['PHIE'] !=0]
test_imp['PHIE'] = test_imp['PHIE'].abs() 

hidden_imp['PHIE'] = hidden_imp.PHIT*(1-hidden_imp.VSHALE)
hidden_imp = hidden_imp[hidden_imp['PHIE'] !=0]
hidden_imp['PHIE'] = hidden_imp['PHIE'].abs() 
#train_imp1.tail()

# # display the log for preview
# plt.figure(figsize=(5,10))
# plt.subplot(122)
# plt.title('PHIE')
# plt.plot('PHIE', 'SI','PI','G','K','VSHALE','DEPTH_MD', data=train_imp)
# plt.gca().invert_yaxis()

In [13]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
x_header=['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO','K','G','VSHALE','PHIT','PHIE','SI','PI',
       'GROUP_encoded', 'FORMATION_encoded', 'WELL_encoded']
y_header=['Lithology_encoded']
x_train = train_imp[x_header]
y_train = train_imp[y_header]
x_test = test_imp[x_header]
y_test = test_imp[y_header]
x_hidden = hidden_imp[x_header]
y_hidden = hidden_imp[y_header]

##Min-Max scaler 
scaler = MinMaxScaler()
x_train_scaled = x_train.copy()
x_test_scaled = x_test.copy()
x_hidden_scaled = x_hidden.copy()

x_train_scaled.iloc[:,:25] = scaler.fit_transform(x_train_scaled.iloc[:,:25])
x_test_scaled.iloc[:,:25] = scaler.transform(x_test_scaled.iloc[:,:25])
x_hidden_scaled.iloc[:,:25] = scaler.transform(x_hidden_scaled.iloc[:,:25])

In [14]:
#Supervised Algorithms
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, precision_score, f1_score
from sklearn.neighbors import KNeighborsRegressor
from pprint import pprint
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
import xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
#Comparing base models accuracies by using k-fold cross validation - 10 folds

from sklearn.model_selection import cross_val_score


model_rf_tuned = RandomForestClassifier(criterion='entropy',max_depth=20, max_features='auto', min_samples_leaf= 1, n_estimators=150)

# Fit the regressor to the training data
model_rf_tuned.fit(x_train_scaled, y_train.values.ravel())

# Prediction
train_pred_rf = model_rf_tuned.predict(x_train_scaled)
open_pred_rf = model_rf_tuned.predict(x_test_scaled)
hidden_pred_rf = model_rf_tuned.predict(x_hidden_scaled)
#Printing Reports 
#Printing Reports


C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [15]:
A = np.load('penalty_matrix.npy')
def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

In [16]:
from sklearn.metrics import classification_report, accuracy_score
print('-----------------------TRAIN SET REPORT---------------------')
print("Open set RMSE:", np.sqrt(mean_squared_error(y_train, train_pred_rf)))
print('Open set penalty matrix score:', score(y_train.values, train_pred_rf))
print('Open set report:', classification_report(y_train, train_pred_rf))
print('-----------------------OPEN SET REPORT---------------------')
print("Open set RMSE:", np.sqrt(mean_squared_error(y_test, open_pred_rf)))
print('Open set penalty matrix score:', score(y_test.values, open_pred_rf))
print('Open set report:', classification_report(y_test, open_pred_rf))
print('-----------------------HIDDEN SET REPORT---------------------')
print("Hidden set RMSE:", np.sqrt(mean_squared_error(y_hidden, hidden_pred_rf)))
print('Hidden set penalty matrix score:', score(y_hidden.values, hidden_pred_rf))
print('Hidden set report:', classification_report(y_hidden, hidden_pred_rf))

-----------------------TRAIN SET REPORT---------------------
Open set RMSE: 0.28163373808912145
Open set penalty matrix score: [-0.04481455]
Open set report:               precision    recall  f1-score   support

         0.0       0.99      0.98      0.99    168937
         1.0       0.98      1.00      0.99    720802
         2.0       0.98      0.95      0.96    150455
         3.0       1.00      0.92      0.96     56320
         4.0       1.00      1.00      1.00     10513
         5.0       1.00      0.75      0.86      1688
         6.0       0.99      0.98      0.99     33329
         7.0       1.00      1.00      1.00      1085
         8.0       1.00      1.00      1.00      8213
         9.0       1.00      0.98      0.99      3820
        10.0       1.00      1.00      1.00       103
        11.0       0.99      0.99      0.99     15245

    accuracy                           0.98   1170510
   macro avg       0.99      0.96      0.98   1170510
weighted avg       0.98      0

C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Open set report:               precision    recall  f1-score   support

         0.0       0.78      0.84      0.81     24048
         1.0       0.84      0.92      0.88     83975
         2.0       0.59      0.31      0.40     17558
         3.0       0.39      0.61      0.47      4798
         4.0       0.00      0.00      0.00       625
         5.0       0.00      0.00      0.00       416
         6.0       0.32      0.05      0.08      3306
         7.0       1.00      0.02      0.03       125
         9.0       0.78      0.56      0.65       690
        11.0       0.72      0.52      0.61      1244

    accuracy                           0.78    136785
   macro avg       0.54      0.38      0.39    136785
weighted avg       0.76      0.78      0.76    136785

-----------------------HIDDEN SET REPORT---------------------
Hidden set RMSE: 1.3206720434045194
Hidden set penalty matrix score: [-0.54030463]
Hidden set report:               precision    recall  f1-score   support

     

In [17]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
facies_colors = ['#F4D03F','#7ccc19','#196F3D','#160599','#2756c4','#3891f0','#80d4ff','#87039e','#ec90fc','#FF4500','#000000','#DC7633']
facies_labels = ['SS', 'S-S', 'SH', 'MR', 'DOL','LIM', 'CH','HAL', 'AN', 'TF', 'CO', 'BS']


#Facies_color_map
facies_color_map = {}
for ind, label in enumerate(facies_labels):
    facies_color_map[label] = facies_colors[ind]
    
def pred_log(logs, well_num, facies_colors, n_pred):
    wells = logs['WELL'].unique()
    logs = logs[logs['WELL'] == wells[well_num]]
    logs = logs.sort_values(by='DEPTH_MD')        #Sorting log by depth
    cmap_facies = colors.ListedColormap(facies_colors[0:len(facies_colors)], 'indexed')
    
    top = logs.DEPTH_MD.min()
    bot = logs.DEPTH_MD.max()
       
    f, ax = plt.subplots(nrows=1, ncols=(12+n_pred), figsize=(15, 12))
    log_colors = ['black', 'red', 'blue', 'green', 'purple','black', 'red', 'blue', 'green', 'purple', 'black', 'red', 'blue', 'green', 'purple', 'black', 'black', 'red', 'blue', 'green', 'purple', 'black', 'black', 'red', 'blue', 'green', 'purple', 'black']

    for i in range(7,18):
      ax[i-7].plot(logs.iloc[:,i], logs.DEPTH_MD, color=log_colors[i])
      ax[i-7].set_ylim(top, bot)
      #ax[i-7].set_xlim(logs.iloc[:,i].min(), logs.iloc[:,i].max())

      ax[i-7].set_xlabel(str(logs.columns[i]))
      ax[i-7].invert_yaxis()
      ax[i-7].grid()

    for j in range((-1-n_pred), 0):
      label = np.repeat(np.expand_dims(logs.iloc[:,j].values, 1), 100, 0)
      im = ax[j].imshow(label, interpolation='none', aspect='auto', cmap=cmap_facies, vmin=0, vmax=12)
      ax[j].set_xlabel(str(logs.columns[j]))

    divider = make_axes_locatable(ax[-1])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax)
    cbar.set_label((12*' ').join(['SS', 'S-S', 'SH', 'MR', 'DOL','LIM', 'CH','HAL', 'AN', 'TF', 'CO', 'BS']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')
        
    f.suptitle('WELL LOGS '+str(wells[well_num]), fontsize=14,y=0.94)

In [18]:
#Storing results
test_rf_tune = test_imp.copy()
hidden_rf_tune = hidden_imp.copy()
#Saving Results
test_rf_tune['RF_ADD'] = open_pred_rf
hidden_rf_tune['RF_ADD'] = hidden_pred_rf

test_rf_tune.to_csv('test_rf_add.csv')
hidden_rf_tune.to_csv('hidden_rf_add.csv')

In [19]:
test_rf_tune = pd.read_csv('test_rf_add.csv')
hidden_rf_tune = pd.read_csv('hidden_rf_add.csv')

In [21]:
x_train_scaled

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,K,G,VSHALE,PHIT,PHIE,SI,PI,GROUP_encoded,FORMATION_encoded,WELL_encoded
0,0.067624,0.073718,0.142953,0.932176,0.660761,0.000637,0.000815,0.000884,0.425020,0.534350,...,0.008466,0.103092,0.534350,0.389828,0.282757,0.232190,0.023277,6.0,-1.0,1.0
1,0.067652,0.073718,0.142953,0.932147,0.660297,0.000637,0.000818,0.000882,0.427069,0.528092,...,0.008479,0.103427,0.528092,0.391988,0.287386,0.233183,0.023562,6.0,-1.0,1.0
2,0.067681,0.073718,0.142953,0.932118,0.660297,0.000637,0.000822,0.000885,0.429527,0.498464,...,0.008491,0.103829,0.498464,0.394580,0.306486,0.234373,0.023849,6.0,-1.0,1.0
3,0.067710,0.073718,0.142953,0.932089,0.659930,0.000637,0.000820,0.000885,0.427843,0.485500,...,0.008491,0.103554,0.485500,0.392804,0.313666,0.233558,0.023744,6.0,-1.0,1.0
4,0.067738,0.073718,0.142953,0.932061,0.659692,0.000637,0.000810,0.000882,0.423503,0.477829,...,0.008491,0.102843,0.477829,0.388228,0.316400,0.231456,0.023461,6.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170506,0.572248,0.355099,0.735606,0.625774,0.234400,0.000637,0.000736,0.000711,0.660253,0.517364,...,0.011803,0.141586,0.517364,0.638733,0.390702,0.346096,0.068605,11.0,7.0,117.0
1170507,0.572277,0.355099,0.735606,0.625774,0.232707,0.000637,0.000736,0.000711,0.663771,0.502079,...,0.011816,0.142162,0.502079,0.642466,0.404586,0.347800,0.068951,11.0,7.0,117.0
1170508,0.572305,0.355099,0.735606,0.625774,0.231589,0.000637,0.000736,0.000711,0.647054,0.442627,...,0.011755,0.139426,0.442627,0.624730,0.444860,0.339705,0.067307,11.0,7.0,117.0
1170509,0.572334,0.355099,0.735606,0.625774,0.230182,0.000637,0.000736,0.000711,0.630860,0.371453,...,0.011695,0.136776,0.371453,0.607554,0.492894,0.331863,0.065714,11.0,7.0,117.0


In [20]:
#Plotting predictions - HIDDEN DATASET
for i in range(1):
  pred_log(hidden_rf_tune, i, facies_colors, 1)

KeyError: 'WELL'

In [ ]:
from sklearn.metrics import confusion_matrix
    from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
    from itertools import product

    def litho_confusion_matrix(y_true, y_pred):
      facies_dict = {0:'Sandstone', 1:'Sandstone/Shale', 2:'Shale', 3:'Marl',
                    4:'Dolomite', 5:'Limestone', 6:'Chalk', 7:'Halite', 
                    8:'Anhydrite', 9:'Tuff', 10:'Coal', 11:'Basement'}

      # creating a lithofacies names
      labels = list(set(list(y_pred.unique()) + list(y_true.unique())))
      label_names = [facies_dict[k] for k in labels]

      # normalizing confusion matrix by the number of predictions
      cm = pd.DataFrame(confusion_matrix(y_true.values, y_pred.values))
      summ = cm.sum(axis=0)
      cm_norm = pd.DataFrame(np.zeros(cm.shape))
      for i in range(cm.shape[1]):
        for j in range(cm.shape[0]):
          cm_norm[i][j] = cm[i][j]*100/summ[i]
      cm_final = cm_norm.fillna(0).to_numpy()

      fig, ax = plt.subplots(figsize=(12,8))
      plt.imshow(cm_final, interpolation='nearest', cmap=plt.cm.Blues)
      plt.title('NORMALIZED CONFUSION MATRIX', size=15)
      tick_marks = np.arange(len(label_names))
      plt.xticks(tick_marks, label_names, rotation=90)
      plt.yticks(tick_marks, label_names)
      plt.colorbar()
      
      # creating a scores format (black and white)
      fmt = '.2f'
      thresh = cm_final.max() / 2.
      for i, j in product(range(cm_final.shape[0]),   range(cm_final.shape[1])):
        plt.text(j, i, format(cm_final[i, j], fmt),
                      horizontalalignment="center",
                      color="white" if cm_final[i, j] > thresh else "black")
              
      plt.ylabel('True label', size=14)
      plt.xlabel('Predicted label', size=14)

In [ ]:
#Confusion Matrix Function
def my_confusion_matrix(y_true, y_pred):
    labels = np.sort(y_true.unique())
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay

    fig, ax = plt.subplots(figsize=(15,10))
    
    cm = confusion_matrix(y_true, y_pred, sample_weight=None, labels=labels, normalize=None)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labels)
    return disp.plot(include_values=True, ax=ax, 
                     xticks_rotation='horizontal', values_format=None)                        #cmap='CMRmap_r'

def default_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(15,10))
    #sns.heatmap(cm, annot=True, linewidths = 0.01)
    sns.heatmap(cm, annot=True)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')